# **Web Scraping for ZipRecruiter**

## **Importing Libraries

In [1]:
#!pip install bs4
#!pip install numpy
#!pip install pandas

#!pip install ast
#!pip install matplotlib
#!pip install nltk
#!pip install wordcloud
!pip install xgboost

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
import nltk
import nltk
import os
import csv
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#from hcluster import pdist, linkage, dendrogram
from nltk.tokenize import RegexpTokenizer
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import pairwise_distances

import scipy.cluster.hierarchy as sch



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Scraping Zip Recruiter for skills in data science job postings



Sraping Zip Recruiter for Job postings to find relevant skills required. Much difficulty scraping this website as it uses an "infinite scroll" system to load more results. Investigated the use of Selenium which can act as a human and click buttons on the page, but browser drivers were needed so considered this not an acceptable option for this project. In the end I decided to make a loop scrolling through 5 pages of each city (20 results per page) to gain the data as typically only 5 pages were available per city.

In [3]:
#Scrapping for 100 data points (20 points per page = 5 pages)
#create an empty DF and set pages = 6
zip_table = pd.DataFrame()
URL = 'https://www.ziprecruiter.com/Jobs/Data-Scientist/--in-Toronto,ON'
#i = 1
for i in range(1,20):
    time.sleep(0.5)
#conducting a request of the stated URL above:
    if i == 1:
        page = requests.get(URL)
        #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
        soup = BeautifulSoup(page.text, 'html.parser')
    elif i > 1:
        URL = 'https://www.ziprecruiter.com/Jobs/Data-Scientist/--in-Toronto,ON/'
        URL+=str(i)
        page = requests.get(URL)
        #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
        soup = BeautifulSoup(page.text, 'html.parser')

    for jobs in soup.find_all("article", class_ = "job_result"): 
      try:
          job_title = jobs.find('span', class_ = 'just_job_title').text.strip()
      #print(job_title)
      except:
          job_title = None

      try:
        location = jobs.find('span', class_ = 'location').text.strip()
      except:
        location = None

      try:
        company = jobs.find('span', class_ = 'name').text.strip()
      except:
        company = None

      try:
        link = jobs.find('p', attrs={'class': 'job_snippet'}).find('a', href=True)['href']
      except:
        link = None

      try:
        salary = jobs.find('span', attrs={'class': 'data_item'}).text.strip()
      except:
        salary = None    
    
      zip_table = zip_table.append({
          'job_title': job_title, 
          'company': company,
          'location': location,
          'salary': salary,
          'link': link}, ignore_index = True)
        


Below, a list of skills was made using terms found in the kaggle data science survey. Previously, links were obained by scraping Zip Recuiter as the job posting descriptions reside in a seprate URL. The new URLs are scraped below, and the regular expressions library is utilyzed to determine if the skills are mentioned in the job postings.

In [4]:

skills = 'deep learning', 'machine learning', 'ai' ,'artificial intelligence', 'ml','data science', 'cost analysis','prediction','code','coding','data architecture','programming','machine learning','ai','statistics','mathematic','communication','critical thinking','visualization','master','professional','bachelor','business decisions','operationalizing','infrastructure','prototype','workflows','modeling','statistical','cloud','jupyter','rstudio','pycharm','atom','matlab','visual studio','spyder','vim','notepad','sublime','google colab','google','google cloud','notebook','ai platform','datalab','binder','ibm watson','ibm watson studio','ocean','python','r','sql','c','c++','java','javascript','typescript','bash','ggplot','matplotlib','plotly','d3','shiny','seaborn','geoplotlib','leaflet','cpu','gpu','tpu','linear','logistic','regression','forest','gradient','bayesian','neural networks','neural','dense','convolutional','transformer','ml','augmentation','automated','feature engineering','selection','architecture','','pipelines','image','video','pil','cv2','skimage','u-net','mask r-cnn','image segmentation','object detection  ','yolov3','retinanet','image classification','vgg','inception','resnet','gan','vae','computer vision','generative networks','word embeddings','vectors','glove','fasttext','word2ve','encoder','decorder seq2seq','vanilla','contextualized embeddings','elmo','cove','gpt-2','bert','xlnet','scikit-learn','tensorflow','keras','randomforest','xgboost','pyTorch','caret','lightgbm','spark mlib','fast.ai','amazon','azure','salesforce','alibaba','aws'
#list of skills]

In [5]:
zip_table['descript']='text'
for skill in skills:
    zip_table[skill] = np.zeros(len(zip_table))
#reset_index
zip_table = zip_table.reset_index()
zip_table.head(5)

,index,company,job_title,link,location,salary,descript,deep learning,machine learning,ai,...,pyTorch,caret,lightgbm,spark mlib,fast.ai,amazon,azure,salesforce,alibaba,aws
0,0,Sodatone,Software Engineer,https://www.ziprecruiter.com/k/l/AAJ5UXbHXY-rj...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,VMware,Salesforce Developer,https://www.ziprecruiter.com/k/l/AAIzXfxumJk78...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Axelon Canada,NAM-CA-Application Programmer-Supervisory (CAD),https://www.ziprecruiter.com/k/l/AAI1Nc5_inAV2...,"Mississauga, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Intact Insurance,"Senior Analyst, Actuarial (Toronto)",https://www.ziprecruiter.com/k/l/AAJ5VwwDfYwbb...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,BlackBerry Corporation,Software Development Student - 4 Month Winter ...,https://www.ziprecruiter.com/k/l/AAKrhIQT_IWvc...,"Mississauga, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
tokenized=[]
for i in range(len(zip_table)):
    detail = requests.get(zip_table.iloc[i,3]) 
    soup = BeautifulSoup(detail.text, "lxml")
    
    #BELOW IS OPTION 2 (CAN NOT HANDLE MULTIPLE WORD SKILLS)
#     try:
#         text = soup.find("div", {'class' : 'job_details_container'}).text.strip().lower()
#       # Remove the html tags
#         re.sub(r'<[^<]+?>','', text)
#   # Tokenize the text
#     except:
#         text = None
#     zip_table.iloc[i,6] = text
#     tokenizer = RegexpTokenizer('\W+',gaps=True)
#     text=tokenizer.tokenize(str(text))
#   # Extract the common skills
#     common_value=list(set(skills).intersection(text))
#     for word in common_value:
#         zip_table.loc[i, word]=1
  #get the text without none
    try:
        text = soup.find("div", {'class' : 'job_details_container'}).text.strip().lower()
        string = re.sub(r'\,', ' ', text) # remove ","
        #string = re.sub(r'\n',' ',string) 
        string = re.sub(r'\,', ' ', string) # remove ","
        string = re.sub('/', ' ', string) # remove "/"
        string = re.sub(r'\(', ' ', string) # remove "("
        string = re.sub(r'\)', ' ', string) # remove ")"
        string = re.sub(' +',' ',string) # remove more than one space 
        print(string)
        
    except:
        text = None
        text = "" 
    #zip_table.iloc[i,6] = string
    for sk in skills :
        if any(x in sk for x in ['+']):
            skk = re.escape(sk)
        else:
            skk = sk
        result = re.search(r'(?:^|(?<=\s))' + skk + r'(?=\s|$)',text)
        
        if result:

            zip_table[sk][i] = 1

save

software engineer

sodatone toronto on



 
 posted: over a month ago
 
 

 full-time
 



 
 job descriptionsodatone helps the music industry find the best undiscovered artists through data analytics and predictive intelligence. we're a toronto-based startup that was acquired by warner music group in 2018 where we're building a large-scale analytics engine to understand how the world interacts with music. there are two core components to our technology. the sodatone platform: the industry's most comprehensive data analytics and visualization platform used by the most influential people in music. they depend on it to find emerging talent that deserves their attention and monitor critical insights and data on their own roster of artists. sodatone ai: using deep learning predictive modelling and our ever-growing database of billions of data points we're developing algorithms that use social streaming and concert data to find the next bruno mars and ed sheeran. help us find and shar

C:\Users\Tyler\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


save

salesforce developer

vmware toronto on



 
 posted: august 19 2020
 
 

 full-time
 



business summary as a member of the vmware salesforce team you will play a highly visible role as a salesforce developer on projects in support of vmware's gtm strategy. this is an internal customer facing role that coordinates with business functional leaders and other global salesforce team members to strategize and execute on key business initiatives. job role and responsibility + design develop test and deploycustom salesforce.com solutions using apex visualforce and or lightning toprovide support for allaspects ofsaas gtm includingcpq quote to cash order management saas business enablement subscription lifecycle business models renewals management gtm offer enablement mergers and acquisitions etc. + collaborate with business stakeholdersand global development teams us and india to define requirements propose ideas and developtechnicalsolutionsin an agile environment + employ declarative

save

senior analyst actuarial toronto 

intact insurance toronto on



 
 posted: june 10 2020
 
 

 full-time
 



 at intact insurance is not about things. it's about people.  it's what we believe and it shapes everything we do.  it's why we strive to provide a second-to-none experience for our customers and create an inspiring workplace for our employees to thrive in. as canada's largest provider of property and casualty insurance we're not only leading our industry but we're redefining what it means to work for it.  as a recognized top employer we're committed to living our values and supporting our dedicated people who bring their best to work each day because they know their work matters.   your job attention analytical superstars!  we're looking for a senior actuarial analyst to join our growing toronto team 700 university avenue . being canada's largest leading provider of property and casualty insurance there's a lot happening behind the scene.  our technology teams reshape t

save

software engineer - remote

triplebyte toronto on



 
 posted: over a month ago
 
 

 $125 000 to $190 000 yearly
 

 full-time
 

 benefits: medical vision 401k dental
 



 
 job descriptionabout triplebytetriplebyte helps companies find and hire great technical talent. for any company building software this is crucial for success. a recent survey of top c-level executives across multiple industries showed that software r&d and recruiting technical talent are the top areas they are planning increased investment and budget over the next 5 years. companies like apple dropbox and american express trust triplebyte’s online technical assessment to identify the best engineers for their open roles and reduce the time and effort it takes to hire them.we just raised a $35 million series b and our team of 40 will double in size during 2019. now is a great time to join as we're on an exciting growth trajectory. you will have lots of opportunity for taking on responsibility and developing

...

save

principal biostatistical programmer

premier research group limited mississauga on



 
 posted: august 15 2020
 
 

 full-time
 



 description position at premier research group limited wish you were here?  recognised recently as a top cro by life science leader premier research is a medium sized full service global cro with circa 1 500 employees worldwide perfectly positioned to be big enough to help yet small enough to care.   premier helps highly innovative bio-pharmaceutical and medical device companies transform life-changing ideas into reality. we take on the most challenging areas of study with a special focus on unmet needs in such areas as oncology neurology paediatrics rare diseases and medical devices supporting the needs of smaller pharmaceutical organisations and biotech companies. there is real opportunity to work across multiple clients on a variety of unique and interesting studies including integrated analyses and regulatory submissions.   continued growth mea

save

software developer 3

oracle toronto on



 
 posted: may 10 2020
 
 

 full-time
 



**software developer 3****preferred qualifications**we're taking on bold challenges and we're looking for an experienced and motivated data processing and streaming pipeline developer to help us build and deliver our high-quality robust and globally scalable big data platform supporting oracle infinity. if you're passionate about using the latest technologies to deliver best-in-class services and working in a fun intense environment where you'll collaborate every day with your team of talented developers architects and designers then please apply.who are we?our development teams focus on enabling our customers to solve their unique business challenges by leveraging the power of our big data and streaming data platforms. our developers design solutions that rely on choosing the right tool for the job often with cutting edge technologies. we offer a highly collaborative work environment where acc

save

digital engagement specialist

talentlab toronto on



 
 posted: over a month ago
 
 

 full-time
 



 
 job descriptiondigital engagement specialist we have partnered with a recognized leader in the canadian fintech to search for digital engagement specialist. this role will focus on working directly with the sales and marketing divisions to develop process improvement initiatives related to the company's digital assets. the right person for this role is passionate about challenging the status-quo and is interested in creating an innovative digital assets portfolio. this is a great role for a digital business analyst data or bi analyst or digital marketing professional to make a huge impact and work directly alongside senior leadership within a national company. **covid update: this role will be 100% remote until our client deems it is safe for their workforce to return to the office. all interviews will be done via video. work will resume in york mills post-covid.  responsibi

save

sap master data lead and application support

crystal claire cosmetics inc. scarborough on



 
 posted: over a month ago
 
 

 full-time
 

 benefits: dental medical vision
 



 
 job descriptionjob descriptionjob title: sap master data lead and application supportdepartment: sapwage category: annual salary with benefitsreport to: director of sapabout the companycrystal group of companies is a leading global player of manufacturing and distributing cosmetics packaging and products. the global head office is located in toronto and the asian head office is located in hong kong. we have cosmetics manufacturing filling facilities in toronto and multiple cosmetics packaging manufacturing facilities across china and taiwan.as our business is fast growing across the world we implemented sap ecc 6.0 as the information management platform for the global head office and overseas companies. the successful candidate will be one of the key members to support and continually enhance the soph

save

reports developer ssrs 

teksystems toronto on



 
 posted: october 07 2020
 
 

 full-time
 



we have a need for a full time salaried report developer\. we are seeking someone with strong experience with ssrs and ssis\. this role will primarily sit remotely\. this person will need to have strong experience pulling data from a variety of data sources using ssis to build their own reports\. job purpose: the report developer's role is to design code test and analyze software programs and applications\. this includes researching designing documenting and modifying software specifications throughout the production life cycle\. the reports developer will also analyze and amend software errors in a timely and accurate fashion and provide status reports where required\. principal responsibilities: \* assist other developers analysts and designers in conceptualizing and developing new software programs and applications\. \* assist in the research preparation and documentation of softw

In [7]:
#zip_table

zip_table = zip_table[zip_table.descript != "none"]
zip_table = zip_table.drop_duplicates(keep='first')
zip_table=zip_table.fillna(0)
#zip_table = zip_table.loc[:, (zip_table != 0).any(axis=0)]
#zip_table.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\ZipRecruiter.csv', index = False)
zip_table.head()

,index,company,job_title,link,location,salary,descript,deep learning,machine learning,ai,...,pyTorch,caret,lightgbm,spark mlib,fast.ai,amazon,azure,salesforce,alibaba,aws
0,0,Sodatone,Software Engineer,https://www.ziprecruiter.com/k/l/AAJ5UXbHXY-rj...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,VMware,Salesforce Developer,https://www.ziprecruiter.com/k/l/AAIzXfxumJk78...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,Axelon Canada,NAM-CA-Application Programmer-Supervisory (CAD),https://www.ziprecruiter.com/k/l/AAI1Nc5_inAV2...,"Mississauga, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Intact Insurance,"Senior Analyst, Actuarial (Toronto)",https://www.ziprecruiter.com/k/l/AAJ5VwwDfYwbb...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,BlackBerry Corporation,Software Development Student - 4 Month Winter ...,https://www.ziprecruiter.com/k/l/AAKrhIQT_IWvc...,"Mississauga, ON",Full-Time,text,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Below, an attempt was made to obtain the most frequently used word across descriptions to mine for skills. Even with the StopWords libary, there were to many general terms used and no skills were obtained. It is clear that a large list of terms needs to be created in order to find the skills with highest importance (frequency). This was done in the above work.

In [8]:
#zip_table.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\ZipRecruiter.csv', index = False)
zip_table = pd.read_csv(r'./ZipRecruiter.csv')
zip_table.head()

# wf = zip_table.descript.str.cat(sep=' ')
# #function to split text into word
# tokens = word_tokenize(wf)
# vocabulary = set(tokens)
# print(len(vocabulary))
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# tokens = [w for w in tokens if not w in stop_words]
# frequency_dist = nltk.FreqDist(tokens)
# sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]

,index,company,job_title,link,location,salary,descript,deep learning,machine learning,ai,...,pyTorch,caret,lightgbm,spark mlib,fast.ai,amazon,azure,salesforce,alibaba,aws
0,0,rockwell automation,Software Data Architect,https://www.ziprecruiter.com/eclk/fAQRWYLiCODH...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,PagerDuty,Senior Data Engineer - Analytics,https://www.ziprecruiter.com/eclk/PrjVFPG3P6gb...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Goodfood Market Corp.,BI Engineer,https://www.ziprecruiter.com/c/Goodfood-Market...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Interac,Senior Data Engineer,https://www.ziprecruiter.com/eclk/gptSw66Ty4w_...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Project X,Consultant- Big Data Developer (4 Positions),https://www.ziprecruiter.com/c/Project-X/Job/C...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


It can be seen that the top 500 words are not relevant towards finding skills wanted in the field of data science.